In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/mulimodal.csv")

In [ ]:
df

,Unnamed: 0,Title,Poem,our_summary
0,0,"Twinkle, Twinkle Little Star","Twinkle, twinkle little star, how I wonder wha...","In the dark and endless night, you shine with ..."
1,1,Wheels on the bus,The wheels on the bus go round and round.\r\nR...,"The doors on the bus go open and shut, open an..."
2,2,Ba Ba Black Sheep,"Baa, baa, black sheep.\r\nHave you any wool ?\...","Baa, baa, black sheep,\nDo you have any wool?\..."
3,3,Jack and Jill,Jack and Jill went up the hill\r\nto fetch a p...,"They both cried out in pain and dismay,\nBut t..."
4,4,Humpty Dumpty,Humpty Dumpty sat on a wall.\r\nHumpty Dumpty ...,"Humpty Dumpty, a popular nursery rhyme charact..."
...,...,...,...,...
165,165,23. Munching Mangoes,\n\nMunching Mangoes\nMunching Mangoes\nMish m...,Mango juice drips\nDown my chin\nSweet and sti...
166,166,24. VEGETABLES,\r\nCarrots and pumpkins\r\nCabbage and peas\r...,Broccoli and cauliflower\nHealthy and deliciou...
167,167,25. I Have Two Arms...,"\r\n\r\nI have two arms that swing like this, ...","I have a body that moves like this, (thrice) I..."
168,168,"26. RABBIT, RABBIT","\n\nRabbit, Rabbit 1, 2, 3\nWill you come and ...","Elephant, Elephant, big and gray,\nWhy do you ..."


In [ ]:
np.mean(np.array([len(i.split(" ")) for i in df["our_summary"]]))

40.631351710395215

In [ ]:
form_summ , topic_summ = [], []

In [ ]:
import os

def read_files_in_directory(directory_path):
    file_contents = []

    # Walk through the directory and its subdirectories
    for root, dirs, files in os.walk(directory_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)

            # Read the content of each file and append it to the list
            with open(file_path, 'r') as file:
                content = file.read()
                file_contents.append(content)

    return file_contents

In [ ]:
form_summ , topic_summ = read_files_in_directory("/content/summary_forms"), read_files_in_directory("/content/summary_topics"),

In [ ]:
all_summ = df["our_summary"].tolist()

In [ ]:
! pip install textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.7 MB/s eta 0:00:00


In [ ]:
import spacy
import pandas as pd
from tqdm import tqdm
import textstat

nlp = spacy.load("en_core_web_sm")


def calculate_abstractness_concreteness(text):

    doc = nlp(text)
    fkgl = textstat.flesch_kincaid_grade(text)
    dcrs = textstat.dale_chall_readability_score(text)
    cli = textstat.coleman_liau_index(text)

    abstract_words = sum(1 for token in doc if token.pos_ in [ 'VERB', 'ADJ', 'ADV'])
    total_words = len(doc)
    abstractness = abstract_words / total_words if total_words > 0 else 0.0

    concreteness = 1.0 - abstractness

    return abstractness, concreteness, fkgl, dcrs, cli

In [ ]:
def calculate_average_scores(texts):
    total_abstractness = 0
    total_concreteness = 0
    total_fkgl = 0
    total_dcrs = 0
    total_cli = 0

    for text in tqdm(texts):
        abstractness, concreteness, fkgl, dcrs, cli = calculate_abstractness_concreteness(text)
        total_abstractness += abstractness
        total_concreteness += concreteness
        total_fkgl += fkgl
        total_dcrs += dcrs
        total_cli += cli

    num_texts = len(texts)
    avg_abstractness = total_abstractness / num_texts
    avg_concreteness = total_concreteness / num_texts
    avg_fkgl = total_fkgl / num_texts
    avg_dcrs = total_dcrs / num_texts
    avg_cli = total_cli / num_texts

    return avg_abstractness, avg_concreteness, avg_fkgl, avg_dcrs, avg_cli

In [ ]:
calculate_average_scores(list(df["our_summary"]))

100%|██████████| 170/170 [00:07<00:00, 23.01it/s]


(0.204218415405429,
 0.7957815845945705,
 8.82176470588235,
 7.45788235294118,
 6.601529411764703)

In [ ]:
calculate_average_scores(form_summ)

100%|██████████| 6321/6321 [01:21<00:00, 77.78it/s]


(0.21127168800387822,
 0.7887283119961307,
 9.0180351210251,
 9.953888625217505,
 11.299346622369853)

In [ ]:
calculate_average_scores(topic_summ)

100%|██████████| 14334/14334 [03:06<00:00, 77.06it/s]


(0.21307783487725362,
 0.7869221651227357,
 9.061448304729941,
 9.692459187944781,
 11.271083437979614)

In [ ]:
summ = form_summ.extend(topic_summ)

In [ ]:
calculate_average_scores(form_summ)

100%|██████████| 20655/20655 [04:11<00:00, 82.17it/s]


(0.2125251040911635,
 0.7874748959088036,
 9.04816267247644,
 9.772463810215571,
 11.2797327523601)

In [ ]:
calculate_average_scores(all_summ)

100%|██████████| 3011/3011 [00:53<00:00, 56.53it/s]


(0.21395661406999766,
 0.7860433859300031,
 8.70820325473265,
 9.601235469943498,
 10.983334440385281)